# Demand side flexibility


In [158]:
using Pkg
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("JuMP")
Pkg.add("Gurobi")
#Pkg.add("NLopt")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


In [159]:
using CSV, DataFrames, JuMP, Gurobi

# Read the CSV data

In [160]:
#fixed_cost = Matrix(CSV.read("/Users/adrienguenard/Desktop/MIT/2.AeroAstro/IDS.522/Project/1.WIP/DSF/test_data/fixed_cost.csv",DataFrame))
#variable_cost = Matrix(CSV.read("/Users/adrienguenard/Desktop/MIT/2.AeroAstro/IDS.522/Project/1.WIP/DSF/test_data/variable_cost.csv",DataFrame))
#demand = Matrix(CSV.read("/Users/adrienguenard/Desktop/MIT/2.AeroAstro/IDS.522/Project/1.WIP/DSF/test_data/demand.csv", DataFrame))
#Availability_matrix = Matrix(CSV.read("/Users/adrienguenard/Desktop/MIT/2.AeroAstro/IDS.522/Project/1.WIP/DSF/test_data/availability.csv", DataFrame));

In [161]:
fixed_cost = Matrix(CSV.read("fixed_cost.csv",DataFrame))
variable_cost = Matrix(CSV.read("variable_cost.csv",DataFrame))
demand = Matrix(CSV.read("demand.csv", DataFrame))
Availability_matrix = Matrix(CSV.read("Availability.csv", DataFrame));

In [162]:
C_FC = fixed_cost[:,1]
C_FOM = fixed_cost[:,2]
C_VOM = variable_cost[:,1]
E_CO2 = variable_cost[:,2]
X_D = demand[1:10]
Availability = Availability_matrix[1:10,1:2]

10×2 Matrix{Float64}:
 0.00393802  0.53339
 0.0         0.513005
 0.0         0.518819
 0.0         0.481426
 0.0         0.475112
 0.0         0.469125
 0.0         0.468391
 0.0         0.453982
 0.0         0.411274
 0.0         0.376903

# Constant

In [163]:
eta = 0.8# Efficiency of the battery

CO2_price = 1000 # price of the CO2 per ton

1000

# Storage model

In [164]:
function storage_charge(x_g, x_d, x_soc, P_s, h, t)
    # Charge of the battery

    if x_g[t] > x_d[t]
        # If the generation is above the demand we can charge
        x_ch_t = minimum([eta * (x_g[t] - x_d[t]), eta * P_s * 1, (1 - x_soc[t])*h*P_s])
        # We charge up to : the energy available above the demand, or the maximum battery capacity, or 
        # the available energy storage left in the battery
    else
        # If we don't have excess energy we don't charge
        x_ch_t = 0
    end
    return x_ch_t
end

function storage_discharge(x_g, x_d, x_soc, P_s, h, t)
        # Discharge of the battery
        if x_g[t] < x_d[t]
            # We discharge if the generation from the other sources is not enought
            x_dch_t = minimum([(x_g[t] - x_d[t]), P_s * 1 , x_soc_t*h*P_s])
        
        else
            # If we already have enough energy we don't discharge
            x_dch_t = 0
    
        end
    return x_dch_t
end

function storage_state_of_charge(x_g, x_d, x_soc, P_s, h, t)
    # Computation of the state of charge 
    if t == 1
        x_soc_t = 1 
    
    else
        # State of charge can be computed from the previous state at t-1
        x_soc_t = x_soc[t-1] + (x_ch[t-1] - x_dch[t-1])/(h*P_s)
    end
        
return x_soc_t
end


storage_state_of_charge (generic function with 1 method)

# Model Functions

In [165]:
function generation_non_storage(P, X, t)

    println("Start generation non storage")
    #X_G_non_storage = ones(4,1)

    # P[1] : Solar 
    println("Generation for solar")
    #Either we generate at the max capacity or at the max availability

    #X_G_non_storage[1] = value(X[t,1])

    # P[2] : Wind 
    println("Generation for wind")
    #Either we generate at the max capacity or at the max availability
    #X_G_non_storage[2] = X[t,2]

    # P[3] : CCGT 
    #print(X_ff[t, 1])
    #X_G_non_storage[3] = X[t, 3]

    # P[4] : Coal 
    #X_G_non_storage[4] =  X_ff[t, 4]
    println(sum(X[t,i] for i in 1:4))
    

    
    return sum(X[t,i] for i in 1:4)
end

function tot_generation(P, X, X_D)

    #P = extract_values(P_var)
    #X_ff = extract_values(X_ff_var)
    println("Start tot generation")
    nb_hours = size(X_D, 1)
    # Initialize the different variable to the number of hours considered
    x_g = zeros(nb_hours,1)
    x_d = X_D
    x_soc = zeros(nb_hours,1)
    x_ch = zeros(nb_hours,1)
    x_dch = zeros(nb_hours,1)
    X_G_tot = zeros(nb_hours,1)
    
    # The battery characteristic are P(5) the power of the battery
    # P(5) the hour of storage of the battery
    println("Size the battery")
    P_s = P[5]
    h = P[6]

    for t=1:nb_hours

        #Generation without storage
        println("Generation non storage")
        #x_g[t] =  generation_non_storage(P, X, t)
        x_g[t] =  sum(X[t,i] for i in 1:4)

        #State of charge of the battery
        println("State of charge")
        x_soc[t] = storage_state_of_charge(x_g, x_d, x_soc, P_s, h, t)
        
        # How much do we charge the battery
        println("Storage charge")
        x_ch[t] = storage_charge(x_g, x_d, x_soc, P_s, h, t)
        
        # How much do we discharge the battery
        println("Storage discharge")
        x_dch[t] = storage_discharge(x_g, x_d, x_soc, P_s, h, t)
        
        # Total generation at time t
        println("Total Generation")
        X_G_tot[t] = x_g[t] + x_dch[t]

    end

    return X_G_tot
end

function supply_demand_equilibrium_constraint(P, X, X_D)

    #P = extract_values(P_var)
    #X_ff = extract_values(X_ff_var)

    println("Call tot_generation function")
    #Generation over all the hours
    X_G_tot = tot_generation(P, X, X_D)
    
    println("initialize nb hours")
    #Number of hours where the supply needs to meet the demand
    nb_hours = size(X_D,1)

    # Compteur of the number of hours where the the supply meets the demand
    compteur = 0 

    println("Start loop over numbers of hours")
    for t = 1:nb_hours

        if X_G_tot[t] >= X_D[t]
            compteur = compteur + 1
        end
    end

    return compteur
end




function cost_mix(P, X)
    
    # The generation for the fossil fuel is for the index 3 and 4 of X
    print(sum((C_FC[i] + C_FOM[i])*P[i] for i in 1:size(P,1)))
    cost = sum((C_FC[i] + C_FOM[i])*P[i] for i in 1:size(P,1)) 
                sum(sum((C_VOM[j] + CO2_price * E_CO2[j])* X[t,j+2] for j in 1:2) for t in 1:size(X,1)) 
    return cost
end

cost_mix (generic function with 1 method)

# Creating the model

List of variable 

In [166]:
Power_generation_ls = 1:5 # Solar, Wind, CCGT, Coal, Battery

fossil_fuel_ls = 1:2 # CCGT, Coal

nb_hours_ls = 1:size(X_D,1)

H = 2

2

In [167]:
model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "NonConvex", 2)
#model = Model(NLopt.Optimizer)
#set_optimizer_attribute(model, "algorithm", :LD_MMA)



#The installed power of the mix
@variable(model, P[Power_generation_ls] >= 0)

@constraint(model, Battery_power, P[5] <= P[1]+ P[2])

@constraint(model, Solar_power, P[1] <= 100000)
@constraint(model, Wind_power, P[2] <= 100000)
#Generation of solar, wind, coal, CCGT
@variable(model, X_gen[nb_hours_ls, Power_generation_ls ] >= 0)

#State of charge of the battery
@variable(model, X_soc[nb_hours_ls] >=0)

# Discharge of the battery
@variable(model, X_dch[nb_hours_ls] >=0)

# Charge of the battery
@variable(model, X_ch[nb_hours_ls] >=0)

# The cost of the mix assuming a certain carbon cost
@objective(model, Min, cost_mix(P, X_gen)) # to be changed


# Cannot produce more than the installed power
@constraint(model, Solar_generation[t in nb_hours_ls], X_gen[t,1] <= P[1])
@constraint(model, Wind_generation[t in nb_hours_ls], X_gen[t,2] <= P[2])
@constraint(model, CCGT[t in nb_hours_ls], X_gen[t,3] <= P[3])
@constraint(model, Coal[t in nb_hours_ls], X_gen[t,4] <= P[4])
@constraint(model, Battery[t in nb_hours_ls], X_gen[t,5] == X_dch[t])

# Cannot produce more than availability for solar and wind 
@constraint(model, Solar_available[t in nb_hours_ls], X_gen[t,1] == Availability[t,1]*P[1])
@constraint(model, Wind_available[t in nb_hours_ls], X_gen[t,2] ==  Availability[t,2]*P[2])

# Supply meets demand at each hours:
@constraint(model, Supply_demand[t in nb_hours_ls], sum(X_gen[t,i] for i in Power_generation_ls) >= X_D[t]);


Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-07
Set parameter NonConvex to value 2
1.64493949138e6 P[1] + 695600.72477 P[2] + 1148000 P[3] + 3518500 P[4] + 434537.234 P[5]

![Alt text](image.png)

Expression for wind and solar

In [168]:
# Constrainhe state of charge
M = 100000000000

#Binary variable for the solar energy
@variable(model, s1[nb_hours_ls], Bin)

# z_1 equal 1 if eta*(x_g_t -x_d) is is greatter than eta*E_max*delta t
println("s_1 expression")
#@constraint(model, s_1_1[t in nb_hours_ls], Availability[t,1] - P[1]   <=  M*s1[t])
#@constraint(model, s_1_2[t in nb_hours_ls],  -Availability[t,1] + P[1] <= M*(1-s1[t]))

#@constraint(model, Solar_prod_1[t in nb_hours_ls], X_gen[t,1] >= P[1] -M*(1-s1[t]))
#@constraint(model, Solar_prod_2[t in nb_hours_ls], X_gen[t,1] >= Availability[t,1] -M*s1[t])


#Binary variable for the wind energy
@variable(model, s2[nb_hours_ls], Bin)

# z_1 equal 1 if eta*(x_g_t -x_d) is is greatter than eta*E_max*delta t
println("s_1 expression")
#@constraint(model, s_2_1[t in nb_hours_ls], Availability[t,2]- P[2]   <=  M*s2[t])
#@constraint(model, s_2_2[t in nb_hours_ls],  -Availability[t,2] + P[2] <= M*(1-s2[t]))

#@constraint(model, Wind_prod_1[t in nb_hours_ls], X_gen[t,2] >= P[2] -M*(1-s2[t]))
#@constraint(model, Wind_prod_2[t in nb_hours_ls], X_gen[t,2] >= Availability[t,2] -M*s2[t]);

s_1 expression
s_1 expression


Create the variable X_min = min(X_1, X_2, X_3)

In [169]:


# X_1, X_2, X_3 = [eta*(sum(X_gen[t,i] for i in Power_generation_ls)- X_D[t]) , eta*P[5], (1- X_soc[t])*P[5]* H]
@variable(model, z1[nb_hours_ls], Bin)
@variable(model, z2[nb_hours_ls], Bin)

# z_1 equal 1 if eta*(x_g_t -x_d) is is greatter than eta*E_max*delta t
println("Z_1 expression")
@constraint(model, z_1_1[t in nb_hours_ls], eta*P[5] - eta*(sum(X_gen[t,i] for i in 1:2)- X_D[t])   <=  M*z1[t])
@constraint(model, z_1_2[t in nb_hours_ls], -eta*P[5] + eta*(sum(X_gen[t,i] for i in 1:2)- X_D[t]) <= M*(1-z1[t]))


#Minimum between X_1 and X_2
@variable(model, min_X_1_X_2[nb_hours_ls])

println("Min X_1, X_2 expression")
@constraint(model, X_1_X_2_lower_1[t in nb_hours_ls], min_X_1_X_2[t] <= eta*(sum(X_gen[t,i] for i in 1:2)- X_D[t]))
@constraint(model, X_1_X_2_lower_2[t in nb_hours_ls], min_X_1_X_2[t] <= eta*P[5])
@constraint(model, X_1_X_2_higher_1[t in nb_hours_ls], min_X_1_X_2[t] >= eta*(sum(X_gen[t,i] for i in 1:2)- X_D[t]) - M*(1-z1[t]))
@constraint(model, X_1_X_2_higher_2[t in nb_hours_ls], min_X_1_X_2[t] >= eta*P[5] - M*z1[t])


# z_2 equal 1 if min(X_1, X_2) is is greater than (1- x_soc[t])*h*E_max
println(" Take the min of min(X_1, X_2) and X_3")
@constraint(model, z_2_1[t in nb_hours_ls], (1- X_soc[t])* P[5] * H - min_X_1_X_2[t] <=  M*z2[t])
@constraint(model, z_2_2[t in nb_hours_ls], - (1- X_soc[t]) * P[5] * H + min_X_1_X_2[t] <= M*(1-z2[t]))


# Constraint for the variable X_ch
println(" Take the overall min")
@variable(model, min_X_1_X_2_X_3[nb_hours_ls])

@constraint(model, X_1_X_2_X_3_lower_1[t in nb_hours_ls], min_X_1_X_2_X_3[t] <= min_X_1_X_2[t])
@constraint(model, X_1_X_2_X_3_lower_2[t in nb_hours_ls], min_X_1_X_2_X_3[t] <= (1- X_soc[t])*P[5]* H)
@constraint(model, X_1_X_2_X_3_higher_1[t in nb_hours_ls], min_X_1_X_2_X_3[t] >= min_X_1_X_2[t] - M*(1-z2[t]))
@constraint(model, X_1_X_2_X_3_higher_2[t in nb_hours_ls], min_X_1_X_2_X_3[t] >= (1- X_soc[t])*P[5]* H - M*z2[t]);



Z_1 expression
Min X_1, X_2 expression
 Take the min of min(X_1, X_2) and X_3
 Take the overall min


Expression of X_ch

In [170]:
@variable(model, Z_X_gen_greater_X_D[nb_hours_ls], Bin)

@constraint(model, Z_X_gen_greater_X_D_1[t in nb_hours_ls], sum(X_gen[t,i] for i in 1:2) - X_D[t] <= M*Z_X_gen_greater_X_D[t])
@constraint(model, Z_X_gen_greater_X_D_2[t in nb_hours_ls], -sum(X_gen[t,i] for i in 1:2) + X_D[t] <= M*(1-Z_X_gen_greater_X_D[t]))

#Expression of X_ch
@constraint(model, X_ch_expression[t in nb_hours_ls], X_ch[t]  == Z_X_gen_greater_X_D[t]*min_X_1_X_2_X_3[t]);

Expression of min of [X_gen[t]- X_D[t],  P[5], X_soc[t]*P[5]* H]

In [171]:
# X_1, X_2, X_3 = [X_gen[t]- X_D[t]) , P[5], X_soc[t])*P[5]* H]
@variable(model, y1[nb_hours_ls], Bin)
@variable(model, y2[nb_hours_ls], Bin)

# z_1 equal 1 if eta*(x_g_t -x_d) is is greatter than eta*E_max*delta t
println("Y_1 expression")
#@constraint(model, y_1_1[t in nb_hours_ls], P[5] - (sum(X_gen[t,i] for i in 1:2)- X_D[t])   <=  M*y1[t])
#@constraint(model, y_1_2[t in nb_hours_ls], -P[5] + (sum(X_gen[t,i] for i in 1:2)- X_D[t]) <= M*(1-y1[t]))


#Minimum between X_1 and X_2
@variable(model, min_extra_prod_P_5[nb_hours_ls])

println("Min X_1, X_2 expression")
#@constraint(model, min_extra_prod_P_5_lower_1[t in nb_hours_ls], min_extra_prod_P_5[t] <= sum(X_gen[t,i] for i in 1:2)- X_D[t])
#@constraint(model, min_extra_prod_P_5_lower_2[t in nb_hours_ls], min_extra_prod_P_5[t] <= P[5])
#@constraint(model, min_extra_prod_P_5_higher_1[t in nb_hours_ls], min_extra_prod_P_5[t] >= (sum(X_gen[t,i] for i in 1:2)- X_D[t]) - M*(1-y1[t]))
#@constraint(model, min_extra_prod_P_5_higher_2[t in nb_hours_ls], min_extra_prod_P_5[t] >= P[5] - M*y1[t])


# y_2 equal 1 if P[5] is is greater than (1- x_soc[t])*h*E_max
println(" Take the min of min(X_1, X_2) and X_3")
#@constraint(model, y_2_1[t in nb_hours_ls], X_soc[t]* P[5] * H - P[5] <=  M*y2[t])
#@constraint(model, y_2_2[t in nb_hours_ls], - X_soc[t] * P[5] * H + P[5] <= M*(1-y2[t]))


# Constraint for the variable X_ch
println(" Take the overall min")
@variable(model, min_extra_prod_P5_soc[nb_hours_ls])

@constraint(model, min_extra_prod_P5_soc_lower_1[t in nb_hours_ls], min_extra_prod_P5_soc[t] <= P[5])
@constraint(model, min_extra_prod_P5_soc_lower_2[t in nb_hours_ls], min_extra_prod_P5_soc[t] <= X_soc[t]*P[5]* H)
@constraint(model, min_extra_prod_P5_soc_higher_1[t in nb_hours_ls], min_extra_prod_P5_soc[t] >= P[5] - M*(1-y2[t]))
@constraint(model, min_extra_prod_P5_soc_higher_2[t in nb_hours_ls], min_extra_prod_P5_soc[t] >= X_soc[t]*P[5]* H - M*y2[t]);



Y_1 expression
Min X_1, X_2 expression
 Take the min of min(X_1, X_2) and X_3
 Take the overall min


Expression of X_dch

In [172]:
@variable(model, Z_X_gen_lower_X_D[nb_hours_ls], Bin)

@constraint(model, Z_X_gen_lower_X_D_1[t in nb_hours_ls], X_D[t] - sum(X_gen[t,i] for i in 1:4)  <= M*Z_X_gen_lower_X_D[t])
@constraint(model, Z_X_gen_lower_X_D_2[t in nb_hours_ls], -X_D[t] + sum(X_gen[t,i] for i in 1:4) <= M*(1-Z_X_gen_lower_X_D[t]))

#Expression of X_dch
# issue with this constraint
@constraint(model, X_dch_expression[t in nb_hours_ls], X_dch[t]  == Z_X_gen_lower_X_D[t]*min_extra_prod_P5_soc[t]);

Expression of state of charge

In [173]:
@constraint(model, SOC_t_1, X_soc[1] == 1)

@constraint(model, SOC_t[t in nb_hours_ls[2:end]], (X_soc[t]-X_soc[t-1])*H*P[5] == (X_ch[t-1]- X_dch[t-1]));

In [174]:
optimize!(model);

Set parameter NonConvex to value 2
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 224 rows, 205 columns and 647 nonzeros
Model fingerprint: 0xcbccd0d4
Model has 89 quadratic constraints
Variable types: 125 continuous, 80 integer (80 binary)
Coefficient statistics:
  Matrix range     [4e-03, 1e+11]
  QMatrix range    [1e+00, 2e+00]
  QLMatrix range   [1e+00, 1e+11]
  Objective range  [4e+05, 4e+06]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+11]
  QRHS range       [1e+11, 1e+11]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 99 rows and 80 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, 

In [175]:
objective_value(model)

LoadError: Result index of attribute MathOptInterface.ObjectiveValue(1) out of bounds. There are currently 0 solution(s) in the model.

In [ ]:
cost_mix(value.(P), value.(X_gen))

5.397194987587383e11

5.397194987587383e11

In [ ]:
println(value.(P))
#println(value.(s1[:]))

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:5
And data, a 5-element Vector{Float64}:
      0.0
      5.094397229564942e-6
 470138.93619790475
      0.0
      0.0


In [ ]:
println(value.(X_gen))

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, 1:10
    Dimension 2, 1:5
And data, a 10×5 Matrix{Float64}:
 0.0  2.7173027849068226e-6  457728.91789728275  0.0       0.0
 0.0  2.613452534541065e-6   448948.56748989015  0.0       0.0
 0.0  2.643070325871118e-6   444216.000397357    0.0      -1.8940772861242294e-7
 0.0  2.4525773082106294e-6  440378.30829733476  0.0       1.8940772861242294e-7
 0.0  2.4204109325897475e-6  442357.4995975796   0.0       0.0
 0.0  2.3899084176704246e-6  450506.1218976101   0.0       0.0
 0.0  2.3861709640869266e-6  466265.8344         0.0       0.0
 0.0  2.3127654479871142e-6  469709.14369768725  0.0       0.0
 0.0  2.095195056968642e-6   470138.93619790475  0.0       0.0
 0.0  1.9200961105525494e-6       0.0            0.0  476334.9041980799


In [ ]:
println(value.(Z_X_gen_greater_X_D))
println(value.(X_soc))

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:10
And data, a 10-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:10
And data, a 10-element Vector{Float64}:
 1.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0


In [ ]:
println(value.(min_extra_prod_P5_soc))

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:10
And data, a 10-element Vector{Float64}:
 -457728.9178935107
 -448948.5674973865
 -444216.00039490336
 -440378.3082975474
 -442357.4995975796
 -450506.1218976101
 -466265.8343948856
 -469709.14369768725
 -470138.93619790475
 -476334.9041980799
